In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
raw_data = pd.read_csv('./dataset.csv', sep=',')
for index in range(len(raw_data)):
    if raw_data.at[index, 'star_rating'] > 3:
        raw_data.at[index, 'star_rating'] = 1
    elif raw_data.at[index, 'star_rating'] == 3:
        raw_data.at[index, 'star_rating'] = 2
    else:
        raw_data.at[index, 'star_rating'] = 0
from bs4 import BeautifulSoup
import re
# data cleaning
raw_data['review_body'] = raw_data['review_body'].str.lower()
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: re.sub(r'http\S+', '', BeautifulSoup(x, 'html.parser').get_text()))
def remove_non_alphabetical(text):
    return re.sub(r'[^a-zA-Z\s]', '', text)

raw_data['review_body'] = raw_data['review_body'].apply(remove_non_alphabetical)
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: ' '.join(x.split()))
contraction_dict = contraction_dict = {
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "can't": "cannot",
    "won't": "will not",
    "isn't": "is not",
    "haven't": "have not",
    "hasn't": "has not",
    "hadn't": "had not",
    "you're": "you are",
    "you've": "you have",
    "you'll": "you will",
    "when's": "when is",
    "let's": "let us",
    "'cause": "because",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "couldn't": "could not",
    "wasn't": "was not",
    "weren't": "were not",
    "I'm": "I am",
    "I've": "I have",
    "I'll": "I will",
    "it's": "it is",
    "that's": "that is",
    "who's": "who is",
    "what's": "what is",
    "where's": "where is",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "she's": "she is",
    "he's": "he is",
    "how's": "how is",
    "you'd": "you would",
    "we'd": "we would",
    "they'd": "they would",
}
raw_data['review_body'] = raw_data['review_body'].replace(contraction_dict, regex=True)
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    words = text.split(" ")
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)
raw_data['review_body'] = raw_data['review_body'].apply(remove_stop_words)
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4') # I have to download this dataset
raw_data['review_body'] = raw_data['review_body'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
import string
def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english') + list(string.punctuation))
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return tokens
raw_data['clean_reviw'] = raw_data['review_body'].apply(preprocess_text)
# better word2vec
# model = Word2Vec(sentences=raw_data['clean_reviw'], vector_size=300, window=5, min_count=1, workers=4)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\monkeydc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\monkeydc\.conda\envs\561\lib\site-packages\ipykernel_launcher.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  app.launch_new_instance()
C:\Users\monkeydc\.conda\envs\561\lib\site-packages\ipykernel_launcher.py:17: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  app.launch_new_instance()
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\monkeydc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\monkeydc\AppData\Roaming\nltk_data...


# HW2-CNN-self-binary

In [2]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('word2vec_model_b.txt', binary=False)

In [3]:
values_to_stratify = [0,1]
def stratified_sample(group, n=100000):
    return group.sample(n=n, random_state=1)

binary_data= raw_data[raw_data['star_rating'].isin(values_to_stratify)].groupby('star_rating', group_keys=False).apply(stratified_sample)

In [4]:
from sklearn.utils import shuffle
shuffled_dataset = shuffle(binary_data, random_state=42)

In [5]:
max_len = 50
def tokenize_and_pad(text, max_len):
    text = text[:max_len]  # Truncate longer reviews
    padding = max(0, max_len - len(text))
    return text + ['<PAD>'] * padding

shuffled_dataset['padded_text_column'] = shuffled_dataset['clean_reviw'].apply(lambda x: tokenize_and_pad(x, max_len))

In [6]:
X_self_wv = np.zeros((len(shuffled_dataset), max_len, 300), dtype=np.float32)

for i, review in enumerate(shuffled_dataset['padded_text_column']):
    for j, word in enumerate(review):
        if word in model:
            X_self_wv[i, j, :] = model[word]
        else:
            X_self_wv[i, j, :] = np.zeros(300, dtype=np.float32)

# X_self_wv = np.array([np.array([model[word] if word in model else np.zeros(300) for word in review]) for review in shuffled_dataset['padded_text_column']])

In [7]:
print(X_self_wv.shape)

(200000, 50, 300)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np

C:\Users\monkeydc\.conda\envs\561\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
X_padded_tensor = torch.FloatTensor(X_self_wv)
X = X_padded_tensor
y = shuffled_dataset['star_rating'].astype(int)  

In [10]:
print(X.shape)

torch.Size([200000, 50, 300])


In [11]:
print(len(X))

200000


In [12]:
X_train_s = X[0:160000]
X_test_s = X[160000:]
y_train_s = y[0:160000]
y_test_s = y[160000:]

In [13]:
class MyCNN(nn.Module):
    def __init__(self, input_size, output_channels):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=300, out_channels=50, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=50, out_channels=10, kernel_size=3)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(10, 2)  

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = self.global_pooling(x).squeeze(2)
        x = self.fc(x)
        return x
input_size = 300  # Size of Word2Vec vectors
output_channels = [50, 10]  # Output channel sizes

cnn_model = MyCNN(input_size, output_channels)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

X_train_tensor_s_cnn = X_train_s.permute(0,2,1)  # Permute dimensions for CNN input
y_train_tensor_s_cnn = torch.LongTensor(y_train_s.values)  # Assuming your target values are integers
X_test_tensor_s_cnn = X_test_s.permute(0,2,1)  # Permute dimensions for CNN input
y_test_tensor_s_cnn = torch.LongTensor(y_test_s.values)

In [14]:
epochs = 5
batch_size = 32

for epoch in range(epochs):
    for i in range(0, len(X_train_tensor_s_cnn), batch_size):
        batch_X = X_train_tensor_s_cnn[i:i + batch_size]
        batch_y = y_train_tensor_s_cnn[i:i + batch_size]

        optimizer.zero_grad()
        outputs = cnn_model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        loss.backward()
        optimizer.step()


In [15]:
from sklearn.metrics import accuracy_score
with torch.no_grad():
    cnn_model.eval()
    outputs = cnn_model(X_test_tensor_s_cnn)
    predictions = torch.argmax(outputs, dim=1)
    accuracy = accuracy_score(y_test_tensor_s_cnn.numpy(), predictions.numpy())
print(accuracy)

0.858525
